In [21]:
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split


## Read table
fname = 'vetting-v01'
fpath = f'../mnt/tess/labels/{fname}.csv'
all_table = pd.read_csv(fpath, header=0, low_memory=False).set_index('Astro ID')

## Rename or drop columns
all_table.drop(columns=['Split'])
all_table = all_table.rename(columns={'filename': 'File', 
                                      'Period': 'Per', 
                                      'Duration': 'Dur',
                                      'Transit_Depth': 'Depth',
                                      'star_rad': 'SRad',
                                      'star_rad_est': 'SRadEst',
                                      'star_mass': 'SMass',})

## Make label columns
disps = ['e', 'p', 'n', 'b', 't', 'u', 'j']
users = ['mk', 'ch', 'et', 'md', 'as', 'dm', 'Tansu', 'Shishir']
for d in disps:
    all_table[f'disp_{d}'] = 0

## Set labels
def set_labels(row):
    a = ~row.isna()
    if row['Final'] == 'i':
        # skip objects labeled as "inside the star"
        return row
    if a['Final']:
        row[f'disp_{row["Final"][0]}'] = 1
        row[f'disp_{row["Final"][1]}'] = 1
    else:
        for user in users:
            if a[user] and row[user]:
                row[f'disp_{row[user][0]}'] += 1
                row[f'disp_{row[user][1]}'] += 1

    return row

all_table = all_table.apply(set_labels, axis=1)

## Only use labelled rows and skip bad rows that have NaN in specific columns
print(f'Total entries: {len(all_table)}')
all_table = all_table[sum(all_table[f'disp_{d}'] for d in disps) > 0]
all_table = all_table[~all_table[['File', 'Per', 'Dur', 'Depth', 'Tmag', 'SRad', 'SRadEst', 'SMass']].isna().any(axis=1)]
print(f'Total labeled entries: {len(all_table)}')

## Train-test split
t_train, t_test = train_test_split(all_table, test_size=0.1, random_state=42)
t_train, t_val = train_test_split(t_train, test_size=1./9, random_state=42)

## Print sizes of arrays and print duplicate counts
print(f'Split sizes. Train: {len(t_train)}; Valid: {len(t_val)}; Test: {len(t_test)}')
print(f'Duplicate TICs: {len(all_table.index.values) - len(set(all_table.index.values))}')
print('Splits')
print('  train:', len(t_train))
print('  val:', len(t_val))
print('  test:', len(t_test))

## Check label arrays
assert not any((t_train['disp_e'] + t_train['disp_p']+ t_train['disp_n'] + t_train['disp_b'] + t_train['disp_t'] + t_train['disp_u'] + t_train['disp_j']) == 0)
assert not any((t_val['disp_e'] + t_val['disp_p']+ t_val['disp_n'] + t_val['disp_b'] + t_val['disp_t'] + t_val['disp_u']+ t_val['disp_j']) == 0)
assert not any((t_test['disp_e'] + t_test['disp_p']+ t_test['disp_n'] + t_test['disp_b'] + t_test['disp_t'] + t_test['disp_u'] + t_test['disp_j']) == 0)

## Save train, test, and validation csv iles
t_train.to_csv(f'../mnt/tess/astronet/tces-{fname}-train.csv')
t_val.to_csv(f'../mnt/tess/astronet/tces-{fname}-val.csv')
t_test.to_csv(f'../mnt/tess/astronet/tces-{fname}-test.csv')
all_table.to_csv(f'../mnt/tess/astronet/tces-{fname}-all.csv')




Total entries: 3859
Total labeled entries: 1997
Split sizes. Train: 1597; Valid: 200; Test: 200
Duplicate TICs: 0
Splits
  train: 1597
  val: 200
  test: 200


In [22]:
all_table[['File', 'Per', 'Dur', 'Depth', 'Tmag', 'SRad', 'SRadEst', 'SMass']].isna().any(axis=1)

Astro ID
1       False
3       False
4       False
5       False
7       False
        ...  
3823    False
3824    False
3825    False
3830    False
3849    False
Length: 1997, dtype: bool